In [1]:
from process import data_process
import numpy as np 
import itertools
from src.preprocess.str_utils import get_extra_chars

In [3]:
df = data_process()

             text_l      title_l
count   5013.000000  5013.000000
mean     816.943347    38.763216
std      979.547978    45.021282
min        0.000000     0.000000
25%      118.000000    12.000000
50%      587.000000    22.000000
75%     1168.000000    36.000000
max    16056.000000   177.000000
302样本无实体
49样本定位的实体未出现在文本中为错误样本
2634样本文本长度超过510
0样本标题长度超过510
切分样本后总共样本数9108


## Sample Split

In [3]:
from sklearn.model_selection import train_test_split
import numpy as np 
train_id, valid_id = train_test_split(np.unique(df['id'].values), test_size=0.2, random_state=1234)
train = df.loc[df['id'].isin(train_id),:]
valid = df.loc[df['id'].isin(valid_id),:]
len(train_id), train.shape, len(valid_id), valid.shape

(3968, (7248, 11), 992, (1860, 11))

In [25]:
valid.to_csv('./trainsample/valid.csv')
train.to_csv('./trainsample/train.csv')

## BIO Tagging 

In [12]:
from src.dataset.converter import single_text
from src.seqlabel_utils import pos2bio
single_text(train['id'].values, train['corpus'].values, 
            train.apply(lambda x: pos2bio(x.corpus, x.ent_pos), axis=1).values,
            './trainsample', 'train_bio')
single_text(valid['id'].values, valid['corpus'].values, 
            valid.apply(lambda x: pos2bio(x.corpus, x.ent_pos), axis=1).values,
            './trainsample', 'valid_bio')

## Span Tagging

In [26]:
from src.dataset.converter import single_text_double_label
from src.seqlabel_utils import pos2span
train_label = train.apply(lambda x: pos2span(x.corpus, x.ent_pos, {'FIN':1}), axis=1).values
valid_label = valid.apply(lambda x: pos2span(x.corpus, x.ent_pos, {'FIN':1}), axis=1).values
single_text_double_label(train['id'].values, train['corpus'].values, 
                         [i[0] for i in train_label], [i[1] for i in train_label],
            './trainsample', 'train_span')
single_text_double_label(valid['id'].values, valid['corpus'].values, 
            [i[0] for i in valid_label], [i[1] for i in valid_label],
            './trainsample', 'valid_span')